In [ ]:
from shapely.geometry import Point # Use in geospatial environment
import pandas as pd
import geopandas as gpd # Use in geospatial environment
import matplotlib.pyplot as plt
import folium #Use in geospatial environment
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head( )

In [ ]:
type(zipcodes)

In [ ]:
zipcodes.info()

In [ ]:
zipcodes.loc[0,'geometry']

In [ ]:
zipcodes.plot();

In [ ]:
#geopandas handles legend styling if you pass a dict of keywords

leg_kwds = {'title': 'Zipcodes', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 3}

zipcodes.plot(column = 'zip', figsize=(10, 10),
              edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds, 
              cmap = 'Set3')
plt.show()

In [ ]:
may_trips_pkl = pd.read_pickle('../data/may_trips.pkl')
print(may_trips_pkl.shape)
may_trips_pkl.head()

In [ ]:
may_trips_pkl.info()

In [ ]:
may_trips_pkl.loc[0].startLatitude

In [ ]:
may_trips_pkl['geometry'] = may_trips_pkl.apply(lambda x: Point((x.startLongitude, x.startLatitude)), 
                                        axis=1)
may_trips_pkl.head(3)

In [ ]:
zipcodes.crs

In [ ]:
may_scooter_geo = gpd.GeoDataFrame(may_trips_pkl, 
                           crs = zipcodes.crs, 
                           geometry = may_trips_pkl['geometry'])

In [ ]:
type(may_trips_pkl)

In [ ]:
type(may_scooter_geo)

In [ ]:
may_scooter_geo.info()

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [ ]:
stops_by_zip = gpd.sjoin(may_scooter_geo, zipcodes, op = 'within')

In [ ]:
stops_by_zip.head()

In [ ]:
stops_by_zip['zip'].value_counts()

In [ ]:
stops_in_37215 = stops_by_zip.loc[stops_by_zip['zip'] == '37215']
stops_in_37215.shape

In [ ]:
polygon37215 = zipcodes.loc[zipcodes['zip'] == '37215']
polygon37215.shape

In [ ]:
#fig, ax = matplotlib.subplots()
ax = polygon37215.plot(figsize = (8, 10), color = 'lightgreen')
stops_in_37215.plot( ax = ax, column = 'tripRoute');
plt.show();

In [ ]:
polygon37215.geometry.centroid

In [ ]:
#find the center of the larger (more southern) polygon and look at it
center = polygon37215.geometry.centroid[38]
print(center)

In [ ]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

In [ ]:
#create our map of Nashville Zip Code 37207 and show it
map_37215 = folium.Map(location =  area_center, zoom_start = 12)
map_37215

In [ ]:
i = 0
for row_index, row_values in stops_in_37215.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
#draw our zip code area: 37215
map_37215 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37215).add_to(map_37215)

#iterate through stops_in_37215 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in stops_in_37215.iterrows():
    loc = [row_values['startLatitude'], row_values['startLongitude']]
    pop = str(row_values['tripRoute'])
    icon=folium.Icon(color="blue",icon="bicycle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_37215)
#map_37215.save('../maps/map37215.html')

#display our map
map_37215

In [ ]:
folium.Map?

In [ ]:
cluster_map_37215 = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map_37215)

folium.GeoJson(polygon37215).add_to(cluster_map_37215)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37215.iterrows():
    loc = [row_values['startLatitude'], row_values['startLongitude']]
    pop = str(row_values['tripRoute'])
    icon=folium.Icon(color="blue",icon="bicyle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
#cluster_map_37215.save('../maps/cluster37215.html')

cluster_map_37215  

In [ ]:
#draw our zip code area: 37215
map_37215 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37215).add_to(map_37215)

#create a list of locations and pass them to FastMarkerCluster()
locations = stops_in_37215[['startLatitude', 'startLongitude']].values.tolist()
map_37215.add_child(
    FastMarkerCluster(locations)
)



#display our map
map_37215